# Project II Phase Transition
## Members and Contributions
俞喆川 2023533165: Implement Part2 and write report
杨子烆 2023533133: Implement Part1 and report examined


## Part 1
We implemented the Monte Carlo method as required by the problem and used the UnionSet algorithm to make efficiency improvements to the process.

In [2]:
import numpy as np
import random

# UnionSet algorithm
class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [0] * size

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)

        if rootP != rootQ:
            if self.rank[rootP] > self.rank[rootQ]:
                self.parent[rootQ] = rootP
            elif self.rank[rootP] < self.rank[rootQ]:
                self.parent[rootP] = rootQ
            else:
                self.parent[rootQ] = rootP
                self.rank[rootP] += 1

    def connected(self, p, q):
        return self.find(p) == self.find(q)

def initialize_grid(size):
    return np.zeros((size, size), dtype=bool)

def initialize_blocked_sites(size):
    return [(i, j) for i in range(size) for j in range(size)]

# Open a site in the grid and connect it to its open neighbors
def open_site(grid, blocked_sites, uf, virtual_top, virtual_bottom):
    size = len(grid)
    blocked_count = len(blocked_sites)
    if blocked_count == 0:
        return None

    rand_index = random.randint(0, blocked_count - 1)
    site = blocked_sites[rand_index]

    blocked_sites[rand_index], blocked_sites[blocked_count - 1] = blocked_sites[blocked_count - 1], blocked_sites[rand_index]
    blocked_sites.pop()

    grid[site] = True

    x, y = site
    index = x * size + y

    if x == 0:
        uf.union(index, virtual_top)
    if x == size - 1:
        uf.union(index, virtual_bottom)

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < size and 0 <= ny < size and grid[nx, ny]:
            neighbor_index = nx * size + ny
            uf.union(index, neighbor_index)

    return site

# Use the UnionFind data structure to check if the virtual top and bottom sites are connected
def percolates(uf, virtual_top, virtual_bottom):
    return uf.connected(virtual_top, virtual_bottom)

# Monte Carlo simulation of percolation
def monte_carlo_percolation(size):
    grid = initialize_grid(size)
    blocked_sites = initialize_blocked_sites(size)
    uf = UnionFind(size * size + 2)
    virtual_top = size * size
    virtual_bottom = size * size + 1
    open_count = 0

    while not percolates(uf, virtual_top, virtual_bottom):
        open_site(grid, blocked_sites, uf, virtual_top, virtual_bottom)
        open_count += 1

    return open_count / (size * size)

def average_percolation_threshold(size, experiments):
    thresholds = [monte_carlo_percolation(size) for _ in range(experiments)]
    return np.mean(thresholds)

# In order not to waste every test result, we manually added history as a past test result
def read_history(threshold, experiments, history):
    return (history[0] * history[1] + threshold * experiments) / (experiments + history[0]), experiments + history[0]
    

size = 20
experiments = 50000
history = (150000, 0.591437)
average_threshold = average_percolation_threshold(size, experiments)

average_threshold, experiments = read_history(average_threshold, experiments, history)

print(f"Average percolation threshold for a {size}x{size} grid over {experiments} experiments: {average_threshold:.6f}")


Average percolation threshold for a 20x20 grid over 200000 experiments: 0.591514


In [22]:
size = 50
experiments = 10000
history = (20000, 0.592551)
average_threshold = average_percolation_threshold(size, experiments)

average_threshold, experiments = read_history(average_threshold, experiments, history)

print(f"Average percolation threshold for a {size}x{size} grid over {experiments} experiments: {average_threshold:.6f}")

Average percolation threshold for a 50x50 grid over 20000 experiments: 0.592551


In [3]:
size = 100
experiments = 5000
history = (10000, 0.592670)
average_threshold = average_percolation_threshold(size, experiments)

average_threshold, experiments = read_history(average_threshold, experiments, history)

print(f"Average percolation threshold for a {size}x{size} grid over {experiments} experiments: {average_threshold:.6f}")

Average percolation threshold for a 100x100 grid over 15000 experiments: 0.592528
